In [1]:
%run training_functions.ipynb

/home/ac.cdavies/.vnv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-11 17:44:50.616087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744393490.630077   82493 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744393490.634394   82493 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744393490.646764   82493 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 

{'record': Value(dtype='string', id=None), 'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
{'text': Value(dtype='string', id=None)}
['train', 'validation', 'test']
['train', 'validation', 'test']
In, Out, get_ipython, exit, quit, open, accelerate, datasets, evaluate, math, np, peft, pickle, pytest, ipytest, pd, transformers, torch, load_dataset, load_dataset_builder, get_dataset_split_names, get_dataset_config_names, LoftQConfig, LoraConfig, get_peft_model, AutoModelForCausalLM, AutoTokenizer, pipeline, TrainingArguments, Trainer, SFTTrainer, SFTConfig, @py_builtins, @pytest_ar, model_from_pkl, ds_builder1, ds_builder2, ds_builder3, ds_gs, ds_gst1_train, ds_gst1_test, ds_gst2_train, ds_gst2_test, search_with_strings, map_data, print_trainable_parameters, metric, comp_metrics_output, compute_metrics, make_trainer, get_dataframe, get_training_output, 

## Dataset value example 
{'record': 'NZ_LIXJ01000001.1', 'text': 'CGACTCCGCACAGGCCCTAGGCGTGGTCCACGCATCGAGCGCGTGCGCCGTGGCATACACCCGGGCCACCCGTACGACCCTTCCGGCCGCCCCGTGTGCGCCCTCGTCGCCGAGTCCGCTCCGGCGCGCGCGGGCGCACTGCTGTTCCGCGCGGTGTCATGCGCTCCTCCCGTCCCGACATGCGCCTTTTTGCAGGGGAGAGGGAGGGGTAGGGTCGCGCGTACGCCTAGCACGGGGGAACGGTCTTGGGGGACGCAGTGGACGTACGGGTGGCGGCGATCGCCAGTCTCACACCGCTCGAAGAACTCGACAGCGATCCGTTCCTCGTGGACACCCGCAGCCAGCACGACATGTGCGCACGCTGGGCCGCGGACAAGGGGTACGTCGTCACCCGCCAGCTCCGCTTCTACGGGCTGCGCCCCGATCACCACGCCCTGTGGAGCGATGTCGAGAGCGGCGACGTCGAGGTGTTCGTCGCCGCCAACGACCGGGTGCTGGAACGGGCCCTCACGTCGGTCGCGGAGTTCGCCGCGGAGTGCGCGCGGCGCGGCGTGCGTCTGGAGATCGCCGGGCTGGACGAGCCGCCGTACAACGCCGGCACGAAGGCGAGCGTGCATCGCAGGCTCTCCATGCCCACCGCCGGTTACGACGGCCGCTGACGCACTCCGCCTCCACTGCCGGCTCGCGGCCCCTGCCCTCGTCCCGGCCCCGTCCCTTCCGCGGCTTGCCGTCGGGCTGTGACAGGCTGGGGGACAGGGTCCGCAGCGGCGGGGCCCGGACGTGAGGTGGTTGAGGCGTGGGTGACGGGCGATGGCGAACGGTCGGCAACGCCCTGATGCGAGTGGCCGTGGTGTGGGCGGTCTCCACGCTCACGATGCTGGTGCTCGCCGGGGTGCTGCCGGAGTTCCGGCTCCAGTCGGACGACGGCGACAGCATCACCAAGACCGCGTTCACCGCGGCCTGGGGCGCCGGAGCGTTCGGCCTGCTCTCCGCACTCGTC'}

In [2]:
ds_gst1_test[0]

{'record': 'JAHFAU010000168.1',
 'text': 'CTCCTCGGCCAGCAAGCGCATCGTGAGAACCGGACTGACGATCAGCAAAACGGTCGCCAGCGGGCCCAATACCATGCGGCCATCCGGGTTCATCTGGCCGGTCGGCAGGCCAAAGTTAACGTTTAGGGAGAAGATCAGGCCGAGCACCAGGAAGCTGAGGGCGGCGGCCGCGTAAGCCAGCGGCGACACGAAGTAGCGGCGGAACTCGCGCACTGCAATAACCCAGCTGGTGCGCACGATCTAGCTCTCCTGCGGCACGACTTGCTCTTCCTCGCGAGTCAGCAGCAGGAAGATGCTTTCCAGGCTGACCCCGGCCGTTGTCAGCTCCAGCAAGTCCAGATCGGCTTCAACGATGCGGCGCGCCAACAGCGGGCGCAGGTCGCGGCCGGGCGAAGAGATCACCTCCACGGTGCCTGCCTGCCCGGCAAGCTCGACCGACTCCACTCCCGGGACGGACTTGAGCAGGTCGACCAGATCGGCTGCCGGGGCAGAGCTGCGGATCAGAATCCGGTACGCACCGGCCAGCTGGGCCGAAAGATTGGCGGGCGTATCTTCCGCCACCAACCGCCCCTTGTTGATAATCAGGACTCGATCACAAACTTGCTGTGCTTCCGAGAGGATGTGGGTGGACAACATGACGGTCCGCTCCCGCCCGAGCTCTCGGATTAAGTCCCGGACTTCGATGATTTGGGCCGGATCCAGTCCAATGGTCGGCTCGTCCAGGATCAAGACCTCGGGCCGGTGCAAGAGTGCCTGGGCCAGGCCGACCCGCTGTCGCAGACCTTTCGACAGGTTGCCAATCAGGCTATCACCGCGTTTTTCGAGGCCGACCTGCGTTAGGGATTCCTCGACCCGGCCATCCAGGTCCTCCACCCGCCGAAGCGAACCCATGAACGAGAGGTATTCGCGGCCGGACATCTCCGGGTACAGCGGGACGGTCTCCGGCAGGTATCCGACC

In [42]:
ds_gst1_test[62] # try prompting it with the newer data

{'record': 'IMGPR_plasmid_3300006450_000026',
 'text': 'CAAAGATCACCAGGAACAGCAGCGCCGGGGCGACCCAAAGCCAGCCCTGCAGGCTTTGCCGCCACGTCGCGCGGCTATGATCGGACCTGTCGGTCATCGGTCAGTCCTGTGGGAGAGGGGGAAATGGGCCGGGCGACCTGCTGGACGCCCGGCGTCGGTCCAGCGTGGGAGCTTACCGCTGGGCCAGCGTTTCCTTGGCCGCGGCGCGCACCTTTTCCATCACGGCTTCCGGCGTCTCATCACCCGACAGAAGGCCCTGTCCCGCGGCGAGGATCACGTTGGCCGAGGCGGTGTCCATGTCCGTGTCCAGCCAGAGCGCCATCCCGGCGGCATCTTCAAGCGCCTTGAGGCCCGCAAGAGTTTCCGGCGTGGTGGTCGCTTCCGTGACCGACCCAATCACCGCGGACGTCATCCCAGTCGTCGACACATACTTTGCCGCTTGTTCCGGCGAGGTGATGAAGTTCAAGAATGCCAGCGCCTCGTCAGGATGCTCTGTCTTGGACGAGATCAGGAATCCGTCCGGCGCCCCGGTCAAAAGGTCGTTGCGGCCCGCGGCACCTTCCATCGGCGGCATCGGGAAGAAGCCCCACCCCTGTTCGGCCACCGGGGTGTCGGCCAGCATGTAGAACTCGACCAGCTCGAGATAGACCATCGCCGACCGGCCGGCGATGAACGCACCCCGCGCTTGCTGATGCGAGATCGCGTTGGCGCCGTTGTTGAAATAGCCCTTGTCGCCGAAGGATTTGTACAGCGCCAAAGCCTCGACATAGCCGGGATGTGTGTAAAGCTGGTCAGCCGGAGCGAGCAGTTGATAGTCGGCCTGACGCACGTCGTCGGGAACCATCTTGGCGAAAAGGTCGCCGATGTAATGCGTTGCCGCCCAAGGCGCCTGGTTGCCGTAGGCGATCGGCACTTCACCCGCCGCCAAGAGCGCGTCGTTAAGC

In [45]:
ds_gst1_train[723]

{'record': 'JAHEFU010000133.1',
 'text': 'TCGTCGCTCATTGCCGGGACATAGGCTCCCCCGGCGGTGCAACTGCCGAGCACGGCGCTTATCTGCGGGATGTTGCGGGCGCTGAGCTGCGCCTGGTTGAAGAAGATCCGCCCGAAGTCGTCATGGTCCGGAAAGACCTCGCTCTGCAGCGGCAGGAAGGCCCCGCCGCTGTCCACGAGGTAGACGCAGGGCAGGTGGTTCTCCAGGGCGATCTCCTGGGCGCGCAGGTGCTTCTTGACCGTCAGCGGGTGGTAGGTCCCGCCCTTGACCGTGGCGTCGTTGGCAACCACCACACAGGTCACGCCGCTGATCACCCCGATGCCGGTCACGATCCCCGCACTGGGCAGCCAGGTCTCATACATCTCCCAGGCGGCCAGCGTGGAGAGTTCCAGCCAGGCCGTGCCGGGGTCCAGCAGCAGGGCCAGGCGCTCGCGCACGAAGAGTTTGCCCTGGCTTTCGTGGCGCTGGCGGTATTTCTCCCCGCCGCCCTCCTTGACCAACGCCAGGCGCTCATCAAGCGTGGCTTCCAGCGCCAGGTTGGCCGCGGCATTGGCCTTGTATTCATCCGAGTTGTGATTGATCCGCGTGGACAGCAGTTCCATGGAGAGTATTCTAAGGGAGGTGCGACACTTGTGTCGCACAAAGGAGGAGTGGTGCAGCTGCGCCACAGGATGCATGGCGCAAGCCAGGAAATGGGAGCGCGGGCATTCAGCCTGGTCTCATTGGAGCGCGAGCATTCAGCTCGCAGGCACGCGATTATTCAACTCGCATGAAATTAAATGCCCTTTCCGTAGCTCCAGAACGCAAGTGTCAGCAACCACAGCATGCAGAGTTCAACCAATAAAGACAGGGAACAGCCGGGTGGGATGTGGTAGGGAATGGCGCAGATCCACATCAGGAACATGATGTTGAGTAGTACCAGTGCACTGCGCATGCCCCAGCGCTTGATATTGCTGTC

In [67]:
# let's try prompting the fine-tuned models
def continued_convo(model, tokenizer, message, convo=None):
    #this will be used to provide models with context from previous conversations
    #this will also allow multiple messages to be consecutively sent to the same model
    pipe=pipeline("text-generation", device_map="auto", max_new_tokens=512, model=model, tokenizer=tokenizer)
    if convo==None:
        messages=[{"role": "system", "content": "You are a helpful assistant"}]
    else:
        messages=convo
    messages.append({"role": "user", "content": message})
    answer=pipe(messages, max_new_tokens=1024)[0]['generated_text'][-1]
    print(answer)
    base_pairs=""
    content=""
    base_pairs=get_pairs(answer) # run the get_pairs method to return just the genome data from a response
    messages.append(answer)
    return pipe, messages, base_pairs

In [68]:
def get_pairs(output):
    base_pairs=""
    content=""
    if ":" in output['content']: #the base pairs are output after a colon
        content=output['content'].split(":")[1] # take all of the string after the colon
        for i in range(len(content)):
            if content[i]=='A' or content[i]=='C' or content[i]=='T' or content[i]=='G': # get the first base pair character
                base_pairs=content[i:] # take the rest of the following pairs
                break # stop the loop when the base pairs are found
        return base_pairs

In [69]:
# this allows prompting where the model is fed some of the training or test data 
# so we see how much more can be extracted
def prompt_with_data(model, tokenizer, genome, pairs):
    pipe=pipeline("text-generation", model=model, tokenizer=tokenizer)
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "What base pairs are present in genome " + genome + "?"},
        {"role": "assistant", "content": "The starting base pairs in JAHEFU010000133.1 are:" + pairs}
    ]
    
    output=pipe(messages, max_new_tokens=512)[0]
    print(output)
    base_pairs=output['generated_text'][-1]['content'].split(":")[1] # extract just the generated pairs
    base_pairs=base_pairs.split(pairs, 1)[1] # split after the base pairs that were fed to the model
    return base_pairs

In [3]:
mis_model=AutoModelForCausalLM.from_pretrained('fine_tuned/mis/')
mis_token=AutoTokenizer.from_pretrained('fine_tuned/mis_token/')

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.05s/it]


In [49]:
model, answer, pairs=continued_convo(mis_model, mis_token, 
                              "What are genome record numbers used for?")
model, answer2, pairs=continued_convo(mis_model, mis_token, 
                               "What would a genome record number look like?", answer)
model, answer3, pairs=continued_convo(mis_model, mis_token, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4, pairs=continued_convo(mis_model, mis_token, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000001.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': " Genome record numbers are used to identify and track specific genetic variations in an individual's genome. These numbers are assigned to specific locations in the genome and are used to identify mutations, insertions, deletions, and other genetic changes that can occur in the DNA sequence. Genome record numbers are used in a variety of applications, including genetic research, medical diagnosis, and personalized medicine. They are also used in forensic science to identify individuals from DNA samples."}


Device set to use cuda:0


{'role': 'assistant', 'content': ' A genome record number is a sequence of letters and numbers that is used to identify a specific location in the genome. The format of a genome record number can vary depending on the specific genome being studied, but it typically includes a combination of letters and numbers that represent the chromosome number, the position on the chromosome, and the specific genetic variation being studied. For example, a genome record number for a mutation on chromosome 1 at position 1000000 might be written as "chr1:1000000" or "1:1000000".'}


Device set to use cuda:0


{'role': 'assistant', 'content': '  The genome identified by the record number NZ\\_LIXJ01000001.1 is the genome of the bacterium Escherichia coli. The record number is a part of the GenBank nucleotide sequence database, which is a public repository of DNA sequences. The record number NZ\\_LIXJ01000001.1 corresponds to a specific DNA sequence for the E. coli genome, which is used for a variety of research and analysis purposes.'}
{'role': 'assistant', 'content': '  The first 100 nucleotides of the sequence of the gene NZ\\_LIXJ01000001.1 are:\n\nATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG

In [50]:
search_with_strings(ds_gst1_train, pairs)

('ATCGATCGATCG', 12, 2)

In [35]:
model, answer, pairs=continued_convo(mis_model, mis_token, "Please provide a list with ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Sure, here are ten example base pairs for a genome:\n\n1. AAGCTGACGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGAT

In [36]:
search_with_strings(ds_gst1_train, pairs)

('AAGCTGACGATCG', 13, 1)

In [37]:
model, answer, pairs=continued_convo(mis_model, mis_token, "Please provide ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Here are ten example base pairs for a genome:\n\n1. AAGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGC

In [38]:
search_with_strings(ds_gst1_train, pairs)

('AAGCTGCTGCTGCTG', 15, 1)

In [54]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=mis_model, tokenizer=mis_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome:CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]['generated_text'][-1]
print(output)
pairs=get_pairs(output)

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are twenty example base pairs for a genome:CGACGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAG'}


In [53]:
search_with_strings(ds_gst1_train, pairs)

('CGACGAGCTGAGC', 13, 1)

In [39]:
model, answer, pairs=continued_convo(mis_model, mis_token, """Please provide five short examples for what
    base pairs of a genome may look like""")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Sure, here are five examples of what base pairs of a genome may look like:\n\n1. AATGCTGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGA

In [40]:
search_with_strings(ds_gst1_train, pairs)

('AATGCTGATGATG', 13, 1)

In [77]:
# prompt with some training data
output=prompt_with_data(mis_model, mis_token, "NZ_LIXJ01000001.1", "CGACTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LIXJ01000001.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:CGACTCAGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTG

('AGCTGCTGCTGCTGC', 15, 1)

In [80]:
output=prompt_with_data(mis_model, mis_token, "JAHFAU010000168.1", "CTCCTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000168.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:CTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCTCCT

('CTCCTCCTCCTCCTCC', 16, 1)

In [16]:
pipe=pipeline("text-generation", model=mis_model, tokenizer=mis_token)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "What base pairs are present in genome JAHFAU010000168.1?"},
    {"role": "assistant", "content": "The starting base pairs in JAHFAU010000168.1 are: CTCCTC"}
]
output=pipe(messages, max_new_tokens=512)[0]
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What base pairs are present in genome JAHFAU010000168.1?'},
  {'role': 'assistant',
   'content': 'The starting base pairs in JAHFAU010000168.1 are: CTCCTCCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTTCTT

In [38]:
search_with_strings(ds_gst1_train, 'CCTGCTGCTGCTGCTGCTGCTGCT') # earlier run

('CCTGCTGCTGCTGCTGC', 17, 1)

In [36]:
search_with_strings(ds_gst1_train, 'CTTCTTCTTCTTCTTCTTCTTCTT')

('CTTCTTCTTCTTCTTCTTC', 19, 1)

In [37]:
search_with_strings(ds_gst1_test, 'CTTCTTCTTCTTCTTCTTCTTCTTCTT')

('CTTCTTCTTC', 10, 7)

In [83]:
# see how much test data the model can extract
output=prompt_with_data(mis_model, mis_token, "JAHFAU010000093.1", "CGATC")
print(search_with_strings(ds_gst1_train, output))
search_with_strings(ds_gst1_test, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHFAU010000093.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:CGATCAGCTGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG

('AGCTGATCGA', 10, 7)

In [84]:
# prompt it with training data from the new half (the second 500)
output=prompt_with_data(mis_model, mis_token, "JAHEFU010000133.1", "TCGTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome JAHEFU010000133.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:TCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTCGTC

('GTCGTCGTCGTCGTCG', 16, 2)

In [64]:
# now prompt with new test data
output=prompt_with_data(mis_model, mis_token, "IMGPR_plasmid_3300006450_000026", "CAAAG")
output

Device set to use cuda:0


{'generated_text': [{'role': 'system',
   'content': 'You are a helpful assistant'},
  {'role': 'user',
   'content': 'What base pairs are present in genome IMGPR_plasmid_3300006450_000026?'},
  {'role': 'assistant',
   'content': 'The starting base pairs in JAHEFU010000133.1 are:CAAAGGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGAT

In [65]:
search_with_strings(ds_gst1_train, 'GATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGAT')

('GATGATGATGATGATGA', 17, 1)

In [66]:
search_with_strings(ds_gst1_test, 'GATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGATGAT')

('GATGATGATGA', 11, 2)

In [22]:
# The model above is fine-tuned using steps as the eval_strategy

In [56]:
mis_epoch=AutoModelForCausalLM.from_pretrained('./fine_tuned/mis2/') 
mis_token_epoch=AutoTokenizer.from_pretrained('./fine_tuned/mis2_token')

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.16s/it]


In [57]:
model, answer, pairs=continued_convo(mis_epoch, mis_token_epoch, 
                              "What are genome record numbers used for?")
model, answer2, pairs=continued_convo(mis_epoch, mis_token_epoch, 
                               "What would a genome record number look like?", answer)
model, answer3, pairs=continued_convo(mis_epoch, mis_token_epoch, 
                               "What genome is identified by a record number of NZ_LIXJ01000001.1?", answer2)
model, answer4, pairs=continued_convo(mis_epoch, mis_token_epoch, 
                               "What are the first 100 nucleotides of the sequence of the gene NZ_LIXJ01000001.1?", answer3)

Device set to use cuda:0
Device set to use cuda:0


{'role': 'assistant', 'content': " Genome record numbers are used to identify and track specific genetic variations in an individual's genome. These numbers are assigned to specific locations in the genome and are used to identify mutations, insertions, deletions, and other genetic changes that can occur in the DNA sequence. Genome record numbers are used in a variety of applications, including genetic research, medical diagnosis, and personalized medicine. They are also used in forensic science to identify individuals from DNA samples."}


Device set to use cuda:0


{'role': 'assistant', 'content': ' A genome record number is a sequence of letters and numbers that is used to identify a specific location in the genome. The format of a genome record number can vary depending on the specific genome being studied, but it typically includes a combination of letters and numbers that represent the chromosome number, the position on the chromosome, and the specific genetic variation being studied. For example, a genome record number for a mutation on chromosome 1 at position 1000000 might be written as "chr1:1000000" or "1:1000000".'}


Device set to use cuda:0


{'role': 'assistant', 'content': '  The genome identified by the record number NZ\\_LIXJ01000001.1 is the genome of the bacterium Escherichia coli. The record number is a part of the GenBank nucleotide sequence database, which is a public repository of DNA sequences. The record number NZ\\_LIXJ01000001.1 corresponds to a specific DNA sequence for the E. coli genome, which is used for a variety of research and analysis purposes.'}
{'role': 'assistant', 'content': '  The first 100 nucleotides of the sequence of the gene NZ\\_LIXJ01000001.1 are:\n\nATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCGATCG

In [25]:
# the above model is fine-tuned using epochs as the eval_strategy

In [58]:
# check if eval_strategy makes a difference in how much data can be extracted
search_with_strings(ds_gst1_train, pairs)

('ATCGATCGATCG', 12, 2)

In [59]:
model, answer, pairs=continued_convo(mis_epoch, mis_token_epoch, "Please provide ten example base pairs for a genome")

Device set to use cuda:0


{'role': 'assistant', 'content': ' Here are ten example base pairs for a genome:\n\n1. AAGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGC

In [60]:
search_with_strings(ds_gst1_train, pairs)

('AAGCTGCTGCTGCTG', 15, 1)

In [62]:
# start off the model's response and see what happens
pipe=pipeline("text-generation", model=mis_epoch, tokenizer=mis_token_epoch)
messages=[
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": """Please provide an example of what the first 
    20 base pairs of a genome may look like"""},
    {"role": "assistant", "content": "Here are twenty example base pairs for a genome: CGAC"}
]

output=pipe(messages, max_new_tokens=512)[0]['generated_text'][-1]
print(output)
pairs=get_pairs(output)

Device set to use cuda:0


{'role': 'assistant', 'content': 'Here are twenty example base pairs for a genome: CGACGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAGCTGAG'}


In [63]:
search_with_strings(ds_gst1_train, pairs)

('CGACGAGCTGAGC', 13, 1)

In [66]:
# prompt with some training data
output=prompt_with_data(mis_epoch, mis_token_epoch, "NZ_LIXJ01000001.1", "CGACTC")
search_with_strings(ds_gst1_train, output)

Device set to use cuda:0


{'generated_text': [{'role': 'system', 'content': 'You are a helpful assistant'}, {'role': 'user', 'content': 'What base pairs are present in genome NZ_LIXJ01000001.1?'}, {'role': 'assistant', 'content': 'The starting base pairs in JAHEFU010000133.1 are:CGACTCAGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTGCTG

('AGCTGCTGCTGCTGC', 15, 1)

In [31]:
# responses between the two models are very consistent